In [ ]:
# 필요 시 주석을 해제하고 아래 라이브러리들을 설치
# !pip install pystan==2.19.1.1
# !pip3 install fbprophet
# !pip install plotly

In [ ]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet

In [ ]:
# 데이터 로드
train = json.load(open('##### train.json 경로#####', 'r', encoding='utf8')) 
submit = json.load(open('##### sample_submission.json 경로 #####', 'r', encoding='utf8')) 

In [ ]:
# 사업장별 dataframe 제작
df_dic = {}
plc_lst = list(train.keys())
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

for i in plc_lst:
    plc_df = pd.DataFrame(columns={'ds', 'pH', 'COD', 'SS', 'N', 'P', 'T'})
    plc_df = plc_df[['ds', 'pH', 'COD', 'SS', 'N', 'P', 'T']]
    
    date_lst = list(train[i].keys())
    plc_df.loc[:, 'ds'] = date_lst

    for index, j in enumerate(date_lst):
        for k in fct_lst:
            try:
                plc_df.loc[index, k] = train[i][j][k]
            except:
                pass
                    
    df_dic[i] = plc_df

In [ ]:
plc_lst[:5]

In [ ]:
df_dic[plc_lst[0]]

In [ ]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
future

In [ ]:
# 추론
start_time = time.time()

for i in plc_lst:
    plc_df = df_dic[i]
    
    for j in fct_lst:
        fct_df = plc_df[['ds', j]]
        fct_df = fct_df.rename(columns={j:'y'})
        model = Prophet()
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

In [ ]:
# 추론 파일 저장
with open('##### prediction.json 경로 #####', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)